# Quantum benchmark

## I - Introduction

## II - Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score, KFold
#Import classical libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

plt.style.use('ggplot')

import functools

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
import logging

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer

from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
import qiskit

In [2]:
import warnings
warnings.filterwarnings('ignore')

## III - Data 

In [3]:
# Read out CSV

df = pd.read_csv('santander_transaction.csv', sep=',')

In [4]:
df.shape

(1000, 201)

In [5]:
%%script false --no-raise-error

import sweetviz as sv

#EDA using Autoviz
sweet_report = sv.analyze(df)

#Saving results to HTML file
sweet_report.show_html('sweet_report.html')

## IV - Modelisation

### Classical

In [6]:

df_labels = df['target']
df.drop(['target'],axis = 1,inplace = True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, df_labels, test_size=0.2, random_state=42)

## Quantum Approaches

In [8]:
np.unique(df_labels)

array([0., 1.])

In [9]:
lda_1 = LDA(n_components=1)
lda_2 = LDA(n_components=1)

In [11]:
lda_1.fit(X_train.iloc[:, :100], y_train)
feature_1 = lda_1.transform(X_train.iloc[:, :100])
lda_2.fit(X_train.iloc[:, 100:], y_train)
feature_2 = lda_2.transform(X_train.iloc[:, 100:])

In [13]:
test_1 = lda_1.transform(X_test.iloc[:, :100])
test_2 = lda_2.transform(X_test.iloc[:, 100:])

In [15]:

features_lda_1 = pd.DataFrame(feature_1)
features_lda_2 = pd.DataFrame(feature_2)
features_lda = features_lda_1.join(features_lda_2, lsuffix="_left", rsuffix="_right")

In [17]:
test_lda_1 =pd.DataFrame(test_1)
test_lda_2 =pd.DataFrame(test_2)
test_lda = test_lda_1.join(test_lda_2, lsuffix="_left", rsuffix="_right")

In [16]:
n_dim = len(features_lda.columns)

## Split train test 

In [19]:
# Split dataset into train and test

#sample_train, sample_test, label_train, label_test = train_test_split(
#     features_lda, y_train, test_size=0.2, random_state=22)

sample_train = features_lda.to_numpy()
sample_test = test_lda.to_numpy()
label_train = y_train
label_test = y_test
# Normalize

std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# Scale for better fit within the feature map

#samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(sample_train)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# Select a sample for a better control of the research and wall time


#test_size = 200 #40
#sample_test = sample_test[:test_size]
#label_test = label_test[:test_size]

In [20]:
# Basic parameters for hybrid model

seed = 8500
feature_dim = n_dim
num_reps = 2
num_shots =256 


## Hybrid

In [21]:
# Define feature_map

feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=num_reps)

# Define the backend
backend = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"), shots=num_shots, seed_simulator=seed, seed_transpiler=seed
)

# Define the kernel

kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

# Model run
svc = SVC(kernel=kernel.evaluate)
#svc.fit(sample_train, label_train)
#score = svc.score(sample_test, label_test)

#print(f"Callable kernel classification test score: {score}")

In [19]:
#result_predict = svc.predict(sample_test)

In [20]:
#print(metrics.classification_report(label_test,result_predict))

In [22]:
from sklearn.model_selection import cross_validate
from tqdm import tqdm
def evaluate_ml_model(_models, X, y, n_fold=10, metric='precision'):
    ''' Function to evaluate a ML and QML model with a list of metrics
    
    
    '''
    results = pd.DataFrame()
    kfold = KFold(n_splits=n_fold)
    columns = []
    for name, model in tqdm(_models):
        # -------------------
        # Variables initialization 
        _df = pd.DataFrame()
        names = []
        means = []
        stds = []
        
        # -------------------
        # k-fold Cross validation
        cv_results = cross_validate(model, X, y, cv=kfold, scoring=metric)
        
        # -------------------
        # Compute the mean and standard deviation 
        for _name, _array in cv_results.items():
            names.append(_name)
            means.append(round(100*_array.mean(), 2))
            stds.append(round(100*_array.std(), 2))
        # -------------------
        # Save the results in a dataframe 
        _df =  pd.DataFrame([means, stds], columns=names)
        columns.extend([name+' mean (%)', name+' std (%)'])
        #results = results.join(_df, on=_df.index)
        results = results.append(_df)
    results.index = columns
    print(results)
    return results

In [43]:
models = []
#models.append(('LR', LogisticRegression(max_iter=1000)))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
models.append(('qsvc', svc))
_metrics = ['precision', 'recall', 'f1', 'accuracy',  'matthews_corrcoef','balanced_accuracy']

In [ ]:
df_results = pd.DataFrame()
df_results = evaluate_ml_model(models, sample_train, label_train, n_fold=10, metric=_metrics)
df_results 

  0%|                                                     | 0/1 [00:00<?, ?it/s]

In [ ]:
j = 0
for i in range(int(len(df_results.index)/2)):

    print(f'{df_results.iloc[j].name.split()[0]} & {df_results.iloc[j][2]} ({df_results.iloc[j+1][2]}) & {df_results.iloc[j][3]} ({df_results.iloc[j+1][3]}) &  {df_results.iloc[j][4]} ({df_results.iloc[j+1][4]}) & {df_results.iloc[j][6]} ({df_results.iloc[j+1][6]}) & {df_results.iloc[j][7]} ({df_results.iloc[j+1][7]}) \\')
    
    j+=2

In [23]:
df_results.to_csv('LDA_fraud_dataset.csv')

## Pennylane

In [26]:
from pennylane import numpy as np

In [27]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [28]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

#print(weights_init, bias_init)

In [29]:
circuit(weights_init, sample_train[0])

tensor(-0.00454489, requires_grad=True)

In [30]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [31]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [32]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [33]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [34]:
Y = np.array(label_train * 2 - np.ones(len(label_train)),requires_grad=True)  # shift label from {0, 1} to {-1, 1}
X = np.array(sample_train, requires_grad=True)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.08535407, requires_grad=True), tensor(0.34006264, requires_grad=True)], Y = -1
X = [tensor(-0.70437879, requires_grad=True), tensor(-0.02989927, requires_grad=True)], Y = -1
X = [tensor(-0.52042413, requires_grad=True), tensor(0.4458723, requires_grad=True)], Y =  1
X = [tensor(-0.11997776, requires_grad=True), tensor(0.30784239, requires_grad=True)], Y =  1
X = [tensor(0.43129711, requires_grad=True), tensor(0.32273232, requires_grad=True)], Y =  1


In [35]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [36]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0
ccost = 0 
for it in range(150):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    '''if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )'''
    prec = metrics.f1_score(Y, predictions, average='binary', pos_label=1)
    if  prec > abest or ((prec == abest) and (cost(weights, bias, X, Y) < ccost)):
        wbest = weights
        bbest = bias
        abest = prec
        ccost = cost(weights, bias, X, Y)
        print('New best')
    #prec = metrics.precision_score(Y, predictions, average='binary')
    print(
        "Iter: {:5d} | Cost: {:0.7f} | f1: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), prec
        )
    )
    
    

New best
Iter:     1 | Cost: 0.9146517 | f1: 0.6062407 
Iter:     2 | Cost: 1.0271006 | f1: 0.1241379 
Iter:     3 | Cost: 0.9447843 | f1: 0.2105263 
New best
Iter:     4 | Cost: 0.7612764 | f1: 0.6434231 
New best
Iter:     5 | Cost: 0.7815182 | f1: 0.7495362 
New best
Iter:     6 | Cost: 0.7493165 | f1: 0.7705882 
New best
Iter:     7 | Cost: 0.7151832 | f1: 0.8191721 
Iter:     8 | Cost: 0.7036907 | f1: 0.7816712 
Iter:     9 | Cost: 0.7175986 | f1: 0.7004471 
Iter:    10 | Cost: 0.7128700 | f1: 0.7004471 
Iter:    11 | Cost: 0.6832924 | f1: 0.8129032 
Iter:    12 | Cost: 0.6780496 | f1: 0.8160000 
Iter:    13 | Cost: 0.7433835 | f1: 0.7655502 
Iter:    14 | Cost: 0.8437661 | f1: 0.7227635 
Iter:    15 | Cost: 0.8274103 | f1: 0.7279215 
Iter:    16 | Cost: 0.7522588 | f1: 0.7662835 
Iter:    17 | Cost: 0.6828545 | f1: 0.8163717 
Iter:    18 | Cost: 0.6808797 | f1: 0.7416667 
Iter:    19 | Cost: 0.7303683 | f1: 0.6356340 
Iter:    20 | Cost: 0.7153637 | f1: 0.6676971 
Iter:    21 | C

In [37]:
Yte = np.array(label_test * 2 - np.ones(len(label_test)))
Xte = np.array(normalize(sample_test))

In [38]:
pd.Series(Yte).value_counts()

 1.0    112
-1.0     88
dtype: int64

In [39]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.7482889902426437, Accuracy: 73.0%


In [40]:
print(metrics.classification_report(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.70      0.69      0.70        90
         1.0       0.75      0.76      0.76       110

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200



In [41]:
print(f'''

    Precision: {round(100*metrics.precision_score(predictions,Yte),2)}%
    Recall: {round(100*metrics.recall_score(predictions,Yte),2)}%
    f1: {round(100*metrics.f1_score(predictions,Yte),2)}%
    Accuracy: {round(100*metrics.accuracy_score(predictions,Yte),2)}%
    Balanced accuracy: {round(100*metrics.balanced_accuracy_score(predictions,Yte),2)}%
    Matthew corcorref: {round(100*metrics.matthews_corrcoef(predictions,Yte),2)}%
    ''')



    Precision: 75.0%
    Recall: 76.36%
    f1: 75.68%
    Accuracy: 73.0%
    Balanced accuracy: 72.63%
    Matthew corcorref: 45.35%
    


In [42]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0
ccost = 1 
for it in range(150):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    '''if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )'''
    prec = metrics.f1_score(Y, predictions, average='binary', pos_label=1)
    if  prec > abest or ((prec == abest) and (cost(weights, bias, X, Y) < ccost)):
        wbest = weights
        bbest = bias
        abest = prec
        ccost = cost(weights, bias, X, Y)
        print('New best')
    #prec = metrics.precision_score(Y, predictions, average='binary')
    print(
        "Iter: {:5d} | Cost: {:0.7f} | f1: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), prec
        )
    )
    
    

New best
Iter:     1 | Cost: 1.0067215 | f1: 0.6343356 
Iter:     2 | Cost: 0.9723392 | f1: 0.4594181 
Iter:     3 | Cost: 0.9288479 | f1: 0.6085754 
Iter:     4 | Cost: 0.8918161 | f1: 0.5893720 
Iter:     5 | Cost: 0.8525129 | f1: 0.6171244 
New best
Iter:     6 | Cost: 0.8058089 | f1: 0.7450980 
Iter:     7 | Cost: 0.7761323 | f1: 0.7324750 
New best
Iter:     8 | Cost: 0.7428069 | f1: 0.7917206 
Iter:     9 | Cost: 0.7662221 | f1: 0.7660415 
Iter:    10 | Cost: 0.8084035 | f1: 0.7406049 
Iter:    11 | Cost: 0.8446540 | f1: 0.7136564 
Iter:    12 | Cost: 0.8359626 | f1: 0.7246637 
Iter:    13 | Cost: 0.7680303 | f1: 0.7643678 
New best
Iter:    14 | Cost: 0.7002655 | f1: 0.8077336 
Iter:    15 | Cost: 0.6693473 | f1: 0.8020050 
Iter:    16 | Cost: 0.6806609 | f1: 0.7362482 
Iter:    17 | Cost: 0.7140716 | f1: 0.6562986 
Iter:    18 | Cost: 0.7155379 | f1: 0.6456693 
Iter:    19 | Cost: 0.7144937 | f1: 0.6425197 
Iter:    20 | Cost: 0.6733834 | f1: 0.7786667 
New best
Iter:    21 | C

KeyboardInterrupt: 

In [45]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.4650416259302474, Accuracy: 86.0%


In [46]:
print(metrics.classification_report(Yte, predictions))

              precision    recall  f1-score   support

        -1.0       0.85      1.00      0.92       154
         1.0       1.00      0.41      0.58        46

    accuracy                           0.86       200
   macro avg       0.93      0.71      0.75       200
weighted avg       0.89      0.86      0.84       200



In [47]:
print(f'''

    Precision: {round(100*metrics.precision_score(predictions,Yte),2)}%
    Recall: {round(100*metrics.recall_score(predictions,Yte),2)}%
    f1: {round(100*metrics.f1_score(predictions,Yte),2)}%
    Accuracy: {round(100*metrics.accuracy_score(predictions,Yte),2)}%
    Balanced accuracy: {round(100*metrics.balanced_accuracy_score(predictions,Yte),2)}%
    Matthew corcorref: {round(100*metrics.matthews_corrcoef(predictions,Yte),2)}%
    ''')



    Precision: 41.3%
    Recall: 100.0%
    f1: 58.46%
    Accuracy: 86.5%
    Balanced accuracy: 92.54%
    Matthew corcorref: 59.28%
    
